# **Lab 2: Regression**

In this lab session, we are given a dataset stored in 'regression-data.txt'. We will develop K Nearest Neighbors (KNNs), linear regression and polynomial regression to fit this dataset. 

**!!! IMPORTANT !!!**
* Please turn OFF any code assistant such as CoPilot;
* You MUST implement KNN, linear regression, and polynomial regression from scratch using the imported libraries below. No additional library is allowed.


In [4]:
# import necessary libraries
import numpy as np
# !pip install matplotlib
import matplotlib.pyplot as plt

### **Load the dataset**
This dataset contains $200$ data points. Each row in the dataset is a data point of the form $[1,~x_1,~x_2,~y]$, where $x_1$ and $x_2$ are two features and $y$ is the target value (i.e., desired output). The first $100$ row will be the training data, next $50$ rows will be the validation data, and the last $50$ rows will be the test data.

Represent the training data as numpy arrays ```Xtrain``` and ```Ytrain```, validation data as ```Xval``` and ```Yval```, and test data as ```Xtest``` and ```Ytest```. The shape of these numpy arrays are 
* ```Xtrain```: (100, 3)
* ```Ytrain```: (100,) not (100, 1)
* ```Xval```: (50, 3)
* ```Yval```: (50,) not (50, 1)
* ```Xtest```: (50, 3)
* ```Ytest```: (50,) not (50, 1)

In [106]:
def read_data(filepath):
    Xtrain, Ytrain, Xval, Yval, Xtest, Ytest = [], [], [], [], [], []
    t_nb = 100
    v_nb = 50
    t_nb = 50
    
    # extract data from filepath
    try:
        with open(filepath, "r") as file:
            data = np.loadtxt(filepath, delimiter=' ', dtype=np.float64)
    except FileNotFoundError:
        print("Error: The file corresponding to the filepath was not found.")

    # divide the data
    Xtrain = data[:t_nb, 0:3]
    Ytrain = data[:t_nb, 3]
    Xval = data[t_nb:t_nb+v_nb, 0:3]
    Yval = data[t_nb:t_nb+v_nb, 3]
    Xtest = data[t_nb+v_nb:, 0:3]
    Ytest = data[t_nb+v_nb:, 3]
    
    return Xtrain, Ytrain, Xval, Yval, Xtest, Ytest


In [107]:
filepath = "regression-data.txt"
Xtrain, Ytrain, Xval, Yval, Xtest, Ytest = read_data(filepath) # Original array has a shape of (200, 4)

### **Implement K Nearest Neighbors**

Let's start with implementing a helper function ```euclidean_distance(Xtrain, Xtest)```. It will compute the Euclidean distance between each test point and all training points and save it into an numpy array ```distances``` of shape (100, 50).

In [ ]:
def euclidean_distance(Xtrain, Xtest):
    

Now follow the instructions to implement your KNN regressor.

In [ ]:
def knn(Xtrain, Ytrain, Xtest, k):
    # compute Euclidean distances between training data and test data
    distances = euclidean_distance(Xtrain, Xtest)

    # Get the indices of the k nearest neighbors
    knn_indices = []
    """
    YOUR CODE HERE
    """

    # Get the labels of the k nearest neighbors
    knn_labels = Ytrain[knn_indices]

    # Return the weighted average among the k neighbors
    # weights should be the reciprocal of the distance
    """
    YOUR CODE HERE
    """

Once we get the predictions, we calculate the mean squared error between predictions and ```Ytest```.

In [ ]:
# get predictions
predictions = knn(Xtrain, Ytrain, Xtest, 5)

# helper function to calculate mean squared error
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# calculate mean squared error
print("Mean Squared Error:", mean_squared_error(Ytest, predictions))

However, it is hard to know the value of ```k``` that could achieve smallest mean squared error on test data. Thus, we will apply the so-called **elbow method** to find the optimal value ```k```.

The elbow method runs like this:
1. Iterate over a list of ```k``` values;
2. For each ```k```, calculate the error (i.e., mean squared error in our case);
3. Find ```k``` such that increasing or decreasing it will cause the error (e.g., mean squared error) to increase

when you draw a curve of ```k``` vs. cost, you will find that the curve actually looks like an elbow -- that's why it is called the **elbow** method.

In [ ]:

k_values = [1, 3, 5, 7, 9]
costs = []

# Now iterate your knn over the k_values and save the mean squared errors into costs
"""
YOUR CODE HERE
"""

We will draw a line of ```k``` vs. costs to help us visualize the change in costs and determine the best ```k``` value.

In [ ]:
def draw_costs_vs_k(k_values, costs):
    plt.figure(figsize=(10, 6))
    plt.plot(k_values, costs, marker='o')
    plt.title('Mean Squared Error vs. k')
    plt.xlabel('Number of Neighbors (k)')
    plt.ylabel('Mean Squared Error')
    plt.grid()
    plt.show()

draw_costs_vs_k(k_values, costs)

***QUESTION***: From the figure drawn by running the code above, could you (1) identify what the best ```k``` value is? If yes, what is the value of ```k``` with the lowest cost? Write your answers in the Markdown cell below.

***Answer:*** YOUR ANSWER HERE 

### **Implement linear regression**

Now we change our regressor from KNN to linear regression. Follow the instructions below to implement linear regression.

$f(x) = w^T x$

where $w[0]$ is the weight for the first column in our dataset $x[0] = 1$.

Linear regression that optimizes the sum-of-square error has a closed-form solution of the optimal $w$, known as the normal equation. Now calculate the optimal weight w using normal equation and (```Xtrain```, ```Ytrain```).

In [52]:
def normal_equation(Xtrain, Ytrain):
    X = Xtrain
    y = Ytrain
    # apply normal equation
    w = np.linalg.inv((X.T @ X)) @ X.T @ y
    return w


def mean_squared_error(Y, X, w): 
    # apply f(x) = wT.x
    y_hat = (w @ X.T)
    # Compute Mean squared error 
    mse = np.mean(np.square(Y - y_hat))
    return mse

Once we have the optimal weight, we can use it make predictions on ```Xtest``` and calculate its mean squared error compared to ```Ytest```.

In [53]:
w_best = normal_equation(Xtrain, Ytrain)
print("Mean Squared Error:", mean_squared_error(Ytest, Xtest, w_best))

Mean Squared Error: 0.7194635198605773


***Your submission will be graded based on the mean squared error you got by running the code above.***

### **Implement polynomial regression**

Sometimes we need a higher order model to more accurately fit the data. To implement polynomial regression, we will first implement a function ```polynomial_feature_transformation``` which takes ```Xtrain``` as input and transforms it into ```XtrainPoly```. Specifically, for our data with features $$[1, x_1, x_2],$$ we will transform it to $$[1, x_1, x_2, x_1 x_2, x_1^2, x_2^2].$$

In [83]:
def polynomial_features(X):
    # create the new features
    interaction = X[:, 1] * X[:, 2]
    quadratic = np.square(X[:, 1:])
    
    # stack the new features with the original array
    XtrainPoly = np.column_stack((X, interaction, quadratic))
    return XtrainPoly

Similarly, once we have the transformed features ```XtrainPoly```, we will use normal equation to compute the optimal weight ```w_best``` and report the mean squared error on test data (```Xtest```, ```Ytest```).

In [84]:
XtrainPoly = polynomial_features(Xtrain)
w_best = normal_equation(XtrainPoly, Ytrain)
XtestPoly = polynomial_features(Xtest)
print("Mean Squared Error:", mean_squared_error(Ytest, XtestPoly, w_best))

Mean Squared Error: 0.7525147352898789


***Your submission will be graded based on the mean squared error you got by running the code above.***